# Introduction to PyTorch

You've implemented a lot of neural network functionalities, such as the forward/backward propagation, ReLU activation, Softmax, Mini-batch SGD, etc., using *Numpy*. You've also put significant effort into making your implementations efficient and fully vectorized in Hand-in assignment 1.

In this notebook, we're going to leave behind your beautiful codebase and instead migrate to a popular and powerful deep learning framework: *PyTorch*. **This make you get started to solve the first part of Hand-in assignment 2.**

### The motivation of using deep learning frameworks

- GPU acceleration: Your code can run on GPUs, enabling much faster training and inference.
- Automatic differentiation: This simplifies the implementation of complex neural networks and differentiable functions.

You can find more tutorials in the official website: https://docs.pytorch.org/tutorials

### Table of contents

This notebook contains two main parts:
1. **Part I, PyTorch Basics.** You will learn and work with PyTorch tensors.
2. **Part II, MNIST Classification.** You will implement a neural network using PyTorch for MNIST classification and aiming to get the same or comparable results to those from Assignment-1.

# Part I: PyTorch Basics

Note: Before running the following cells, you can manually switch to a GPU device on Colab by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`.

In [ ]:
import math
import numpy as np
import torch
import matplotlib.pyplot as plt

print("PyTorch version: ", torch.__version__)

**Numpy array vs. Pytorch tensor**

PyTorch tensors are a specialized data structure that are very similar to NumPy’s ndarrays, except that tensors can run on GPUs and support automatic differentiation. If you’re already familiar with ndarray, you’ll be right at home with the Tensor API. If not, no worries—just follow along!

**Initialize a tensor**

In [ ]:
data = [[1, 2], [3, 4]]
data_numpy = np.array(data)
print("data:", data)

# directly from the data
tensor_from_data = torch.tensor(data)
print(tensor_from_data)

In [ ]:
# from the Numpy ndarrays
tensor_from_numpy = torch.from_numpy(data_numpy)
print(tensor_from_data)

In [ ]:
# initialize with random or constant values
rand_tensor = torch.rand(tensor_from_data.shape)
print(rand_tensor)
zero_tensor = torch.zeros(tensor_from_data.shape)
print(zero_tensor)
ones_tensor = torch.ones(tensor_from_data.shape)
print(ones_tensor)

**Tensor to numpy**

In [ ]:
numpy_from_tensor = tensor_from_data.numpy()
numpy_from_tensor

**Attributes of a Tensor** 

Tensor attributes describe their shape, datatype, and the device on which they are stored.

In [ ]:
tensor1 = tensor_from_numpy.float()
tensor2 = rand_tensor.float()
tensor = torch.mul(tensor1, tensor2)
print(tensor)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

**Tensor manipulation on the GPU deivce**

Note: the following example will be executed on GPU only if you have access to a GPU.

In [ ]:
print("Use GPU: ", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
tensor1_gpu = tensor1.to(device)
tensor2_gpu = tensor2.to(device)
tensor_gpu = torch.mul(tensor1_gpu, tensor2_gpu)
print("tensor device: ", tensor_gpu.device)

You can also manually convert the tensor back to CPU.

In [ ]:
tensor_cpu = tensor_gpu.cpu()
print("tensor device: ", tensor_cpu.device)

**Back-propagation with PyTorch's automatic differentiation engine**

Recall that training a neural network typically happens in three steps: *Forward Propagation*, *Backward Propagation*, and *Gradient Descent Optimization*. 

During the forward pass in our model, PyTorch defines a computational graph, whose nodes are tensors and whose edges are the functions mapping one input tensor to another output tensor. By backpropagating this graph, we obtain the desired gradients which can be used to optimzie the parameters.

In [ ]:
# state that we want to obtain the gradients of x
x1 = torch.tensor([2., 3.], requires_grad=True)
x2 = torch.tensor([1., 2.], requires_grad=True)

# run the forward pass and build the computational graph
z = torch.sum(x1 - x2)
print(f"Output of the forward pass: {z}\n")

# backpropagate the graph
z.backward()

print("First computation")
print(f"Gradient information of x1: {x1.grad}")
print(f"Gradient information of x2: {x2.grad}\n")

# Repeat the computations and backpropagate again, the gradients will be accumulated.
z = torch.sum(x1 - x2)

# backpropagate the graph
z.backward()

print("Second computation")
print(f"Gradient information of x1: {x1.grad}")
print(f"Gradient information of x2: {x2.grad}")

To override the gradients, we have to set them to zero before running the forward pass.

In [ ]:
# set the gradient of x to zero
x1.grad.zero_()
x2.grad.zero_()

# run the forward pass and build the computational graph
z = torch.sum(x1 - x2)

# backpropagate the graph
z.backward()

print(f"Gradient information of x1: {x1.grad}")
print(f"Gradient information of x2: {x2.grad}")

If we want perform operations that should not be tracked by PyTorch, we can wrap them inside a torch.no_grad() clause.

In [ ]:
with torch.no_grad():
    z = torch.sum(x1 - x2)
print(z)

### Linear regression example

Now we do something (slightly) more interesting. We consider linear regression using PyTorch. We study the model
\begin{equation*}
    y = \beta + \omega x + \epsilon,
\end{equation*}
where $\epsilon$ represents the noise term.

We generate training data $\{x_i, y_i\}_{i=1}^I$ with $I = 100$ data points from the model with the true parameters $\boldsymbol{\phi} = \begin{bmatrix} \beta & \omega\end{bmatrix}^\intercal = \begin{bmatrix} 0.3 & 0.1 \end{bmatrix}^\intercal$ and normally distributed noise $\epsilon \sim \mathcal{N}(0, 0.01^2)$. The inputs $x_i$ are sampled uniformly from the interval $[0,1]$.

We train the model on CPU by default.

In [ ]:
phi_true = torch.tensor([0.3, 0.1])
n = 100

torch.manual_seed(1234)
x = torch.rand(n)
y = phi_true[0] + phi_true[1] * x + 0.01 * torch.randn(n)

We plot the data to see what it looks like.

In [ ]:
plt.scatter(x, y, label='data')
plt.plot([0, 1], [phi_true[0], phi_true[0] + phi_true[1]], 'k--', label='true model')
plt.legend()
plt.show()

We want to estimate the parameters $\boldsymbol{\phi} = \begin{bmatrix} \beta & \omega\end{bmatrix}^\intercal$ of the linear regression model based on the training data $\{x_i,y_i\}_{i=1}^I$. In addition, we proceed by implementing the mean squared error cost function $L[\boldsymbol{\phi}]$, where
\begin{equation*}
L[\boldsymbol{\phi}] = \frac{1}{I} \sum_{i=1}^I {(\beta + \omega x_i − y_i)}^2 = \frac{1}{I} {\|\mathbf{X} \boldsymbol{\phi} − \mathbf{y} \|}^2_2.
\end{equation*}
We multiply the input $\mathbf{X}$ with the variable $\boldsymbol{\phi}$, subtract the corresponding outputs, take the square, and finally average over all data samples.

Note: starting from Python 3.5, many NumPy-style operators—such as `@` for matrix multiplication and `**` for element-wise exponentiation—are supported at the language level and can be directly applied to tensor matrices as well. 

In [ ]:
# define the initial model parameters as a column vector
phi = torch.ones(2, 1, requires_grad = True)

X = torch.stack([torch.ones(n), x], dim=1)
Y = y.view(-1, 1)

training_loss = []
for i in range(1000):
    # TODO -- compute the mean squared error (as loss) for the current parameters
    # Replace this line!
    loss = (X @ phi).mean()
    # perform backpropagation
    loss.backward()
    training_loss.append(loss.item())
    if i == 0 or (i + 1) % 100 == 0:
        print(f"Epoch {i + 1:04d}: training loss {training_loss[-1]: 9.6f}")
    
    # perform a gradient descent step
    with torch.no_grad():
        phi -= 0.1 * phi.grad
    # reset the gradient information
    phi.grad.zero_()
    
plt.figure()
plt.plot(np.arange(1, len(training_loss) + 1), training_loss, 'o', label='training loss')
plt.yscale('log')
plt.xlabel('number of iterations')
plt.legend()
plt.show()


In [ ]:
# we have to detach from the graph since otherwise the automatic conversion
# to a numpy array performed by matplotlib will error
phi_numpy = phi.detach().numpy()
print(phi_numpy)
plt.scatter(x, y, label = 'data')
plt.plot([0, 1], [phi_true[0], phi_true[0] + phi_true[1]], 'k--', label = 'true model')
plt.plot([0, 1], [phi_numpy[0], phi_numpy[0] + phi_numpy[1]], 'r-', label = 'estimated model')
plt.legend()
plt.show()

# Part II: Classification of hand written digits with PyTorch

In Hand-in Assignment 1, you implemented a neural network for MNIST classification using NumPy. In this task, you will re-implement the same model using the PyTorch API. By keeping all hyperparameters identical, you should expect to obtain the same results as in your NumPy-based implementation.

### Step-1: Load and normalize the MNIST training and test datasets using `torchvision`.

In this step, we use the `torchvision` dataset and transform APIs to load and normalize MNIST, and the `DataLoader` to efficiently batch and iterate over the data during training and evaluation. Specifically, `torchvision.datasets.MNIST` provides a ready-to-use wrapper that automatically handles downloading, caching, and accessing the dataset, returning image–label pairs in a PyTorch-friendly format.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torchvision.transforms import v2 as T


seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

# T.Compose takes a list of transformations and applies them in sequence
transform = T.Compose([
    T.PILToTensor(),
    T.ToDtype(torch.float32, scale=True), # scale=True scales values to the interval [0,1] if needed
])
# Notice the new keyword argument added at the end!
trainset = MNIST(
    './data',
    train=True,
    download=True,
    transform=transform)

testset = MNIST(
    './data',
    train=False,
    download=True,
    transform=transform)

# extract a complete PyTorch dataset
def extract(dataset):
    datasize = len(dataset)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=datasize, shuffle=False)
    return next(iter(dataloader))

# extract all test images and labels into PyTorch tensors
# the training data will be loaded in batches during training
test_X, test_Y = extract(testset)

# Notice how image is now a tensor
print(test_X.shape)

### Step-2: Implement a multi-layer neural network

PyTorch provides the `torch.nn.Module` API to define arbitrary network architectures while tracking all learnable parameters automatically. **You should use this API to implement exactly the same fully-connected network as you designed for Assignment 1.**

Note: The input data $X$ are grayscale images of $28\times 28$ pixels. The first dimension will be the number of data points that are provided to the network. The input data is flattend into a matrix with $28 \times 28 = 784$ columns using `X.view(-1, 784)`, where each colum represents one pixel.

In [ ]:
def weight_init(net_l):
    # network initialization function
    if not isinstance(net_l, list):
        net_l = [net_l]
    for net in net_l:
        for m in net.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight)
                m.bias.data.fill_(0.0)

class Net(nn.Module):
    def __init__(self, activation="sigmoid"):
        super(Net, self).__init__()
        U1 = 250 # number of unit for hidden layer 1
        
        # define your neural layers here
        # Implement the same fully-connected network as you designed for Assignment 1!
        # TODO -- Replace the lines below with your network setting!
        self.fc1 = nn.Linear(784, U1)
        self.fc2 = nn.Linear(U1, 10)
        # define the activation function
        if activation == "sigmoid":
            self.activate = nn.Sigmoid()
        elif activation == "relu":
            self.activate = nn.ReLU()
        
        # network initialization
        self.apply(weight_init)
        
    def forward(self, input):
        # flatten the data into a matrix with 28 x 28 = 784 columns
        x = input.view(-1, 784)
        
        # compute the hidden layers and predictions with
        # e.g., self.fc1, self.fc2, self.fc3
        # TODO -- Replace the lines below with your network setting!
        x = self.activate(self.fc1(x))
        x = self.fc2(x)
        
        return x


**TODO:**  Compute how many parameters of this neural network?

### Step-3: Loss function and predict function

We define the cross-entropy for the predicted probabilities $G$ (10-dimensional vectors) and the labels $Y$ (integers between 0 and 9). In addition, we use the `argmax` function to take final predictions.

Note: In PyTorch, we use `torch.nn.functional.cross_entropy` which internally applies a softmax to the model logits, so the input should be raw, unnormalized scores rather than probabilities.

In [ ]:
def cross_entropy(G, Y):
    # TODO -- implement the cross entropy loss
    # use F.cross_entropy
    # Replace this line!
    loss = G.mean()
    return loss


In [ ]:
def predict(net, X):
    return net(X).argmax(dim=1)

def accuracy(G, Y):
    return (G.argmax(dim=1) == Y).float().mean()

### Step-4: Train the network

You should adopt the same settings e.g., batch size, the number of iterations, optimizer, learning rate, etc., as you did in Assignment-1 to train the network.

In [ ]:
# initialize the test and training error statistics
test_accuracy = []
test_costs = []
test_iter = []
train_accuracy = []
train_costs = []

# initialize the neural network and move it to the GPU if needed
net = Net(activation="relu")
net.to(device)

# define the optimization algorithm
learning_rate = 0.1
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

# define the data loader for batches of the training data
batch_size = 100
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=2, shuffle=True)

# perform multiple training steps
num_epochs = 5

t = 0 # current iteration
done = False
start_time = time.time()
for epoch in range(num_epochs):
    for (batch_X, batch_Y) in trainloader:
        # move batch to the GPU if needed
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # TODO -- Network forward, loss calculation, and backward!
        # Replace the lines below!
        batch_G = batch_Y
        loss = (batch_G - batch_Y).mean()
        # perform gradient descent step
        optimizer.step()
        
        # don't bother too much about the following lines!
        with torch.no_grad():
            # evaluate the performance on the training data at every 10th iteration
            if t % 100 == 0:
                train_costs.append(loss.item())
                train_accuracy.append(accuracy(batch_G, batch_Y).item())
                
            # evaluate the performance on the test data at every 100th iteration
            if t % 100 == 0:
                # move test data to the GPU if needed
                X, Y = test_X.to(device), test_Y.to(device)

                # compute predictions for the test data
                G = net(X)
                test_costs.append(cross_entropy(G, Y).item())
                test_accuracy.append(accuracy(G, Y).item())
                test_iter.append(t)

                # print the iteration number and the accuracy of the predictions
                print(f"Epoch {epoch:2d} - Step {t:5d}: train accuracy {100 * train_accuracy[-1]:6.2f}% " \
                      f"train costs {train_costs[-1]:5.2f}  " \
                      f"test accuracy {100 * test_accuracy[-1]:6.2f}% " \
                      f"test costs {test_costs[-1]:5.2f}")
            
        # accumulate iterations
        t += 1
            
end_time = time.time()
elapsed = end_time - start_time


### Step-5: Evaluation on the test data

In [ ]:
# plot the cross-entropy
def training_curve_plot(title, 
                        train_costs, test_costs, train_accuracy, test_accuracy, 
                        batch_size, learning_rate, num_epochs, elapsed):
    # Plot training curves in a format recomended for Hand-in assignment 1 and 2
    #
    # Input
    # title - title for the plot
    # train_costs - Array of training costs
    # test_costs - Array of test costs
    # train_accuracies - Array of training accuracies
    # test_accuracies - Array of test accuracies
    # batch_size - batch size for training data used during training
    # num_epochs - Number of epochs used during training
    # elapsed - Time elapsed in seconds during training

    lg=18
    md=13
    sm=9
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(title, y=1.15, fontsize=lg)    
    elapsed_min, elapsed_sec = divmod(elapsed, 60)
    sub = f'|  Batch size:{batch_size}  |  Learning rate:{learning_rate} | Number of Epochs:{num_epochs} ' \
                + f'| Training time: {elapsed_min:.0f} min {elapsed_sec:.1f} sec |'
    fig.text(0.5, 0.99, sub, ha='center', fontsize=md)
    x = np.array(range(1, len(train_costs)+1))*num_epochs/len(train_costs)
    axs[0].plot(x, train_costs, label=f'Final train cost: {train_costs[-1]:.4f}')
    axs[0].plot(x, test_costs, label=f'Final test cost: {test_costs[-1]:.4f}')
    axs[0].set_title('Costs', fontsize=md)
    axs[0].set_xlabel('Epochs', fontsize=md)
    axs[0].set_ylabel('Cost', fontsize=md)
    axs[0].legend(fontsize=sm)
    axs[0].tick_params(axis='both', labelsize=sm)
    # Optionally use a logarithmic y-scale
    #axs[0].set_yscale('log')
    axs[1].plot(x, train_accuracy, label=f'Final train accuracy: {100*train_accuracy[-1]:.2f}%')
    axs[1].plot(x, test_accuracy, label=f'Final test accuracy: {100*test_accuracy[-1]:.2f}%')
    axs[1].set_title('Accuracy', fontsize=md)
    axs[1].set_xlabel('Epochs', fontsize=md)
    axs[1].set_ylabel('Accuracy (%)', fontsize=sm)
    axs[1].legend(fontsize=sm)
    axs[1].tick_params(axis='both', labelsize=sm)

title = "Training curve of the fully-connected network"
training_curve_plot(title, 
                    train_costs, test_costs, train_accuracy, test_accuracy, 
                    batch_size, learning_rate, num_epochs, elapsed)

In [ ]:
# evaluate the network on 100 random test images
with torch.no_grad():
    # obtain 100 random samples from the test data set
    random_X, random_Y = next(iter(torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True)))
    
    # move data to the GPU if needed
    random_X, random_Y = random_X.to(device), random_Y.to(device)
    
    # compute the predictions for the sampled inputs
    random_Yhat = predict(net, random_X)

    # sort the predictions with the incorrect ones first
    indices_incorrect_first = (random_Yhat == random_Y).float().argsort()

# plot the images
num_rows = 10
num_cols = 10
num_images = num_rows * num_cols
plt.figure(figsize=(num_cols, num_rows))

for i, index in enumerate(indices_incorrect_first, 1):
    plt.subplot(num_rows, num_cols, i)
    plt.xticks([])
    plt.yticks([])
    
    # plot the image
    plt.imshow(random_X[index, :, :].view(28, 28).cpu().numpy(), cmap=plt.cm.binary)
    
    # add the prediction as annotation (incorrect predictions in red, correct ones in blue)
    color = 'blue' if random_Yhat[index] == random_Y[index] else 'red'
    plt.text(0, 25, random_Yhat[index].item(), fontsize=25, color=color)
        
plt.show()